In [1]:
%load_ext autoreload

In [2]:
%autoreload

from allennlp.commands.elmo import ElmoEmbedder
import os
from sys import path
path.append('../..')
from relation_extraction.data import utils
import h5py
import numpy as np
dataset = 'ddi' #TODO: update this
data_path = '/data/medg/misc/geeticka/relation_extraction/' + dataset + '/'
def res(path): return os.path.join(data_path, path)
out_path = 'pre-processed/original/elmo/'

[nltk_data] Downloading package wordnet to
[nltk_data]     /afs/csail.mit.edu/u/g/geeticka/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
if not os.path.exists(res(out_path)):
    os.makedirs(res(out_path))
if not os.path.exists(res(out_path + 'input-sentences')):
    os.makedirs(res(out_path + 'input-sentences'))

In [4]:
elmo = ElmoEmbedder()

## Getting all of the ELMo layers

In [5]:
border_sizes = [-1, 50, 1, 20]

In [6]:
def get_sentences(filename, border_size):
    data = open(res('pre-processed/original/' + filename))
    data = utils.split_data_cut_sentence(data, border_size)
    sentences = data[0]
    return sentences

In [7]:
def write_sentences_to_txt(sentences, filename):
    with open(res(out_path + 'input-sentences/'+filename + '.txt'), 'w') as f:
        for sentence in sentences:
            sentence_string = " ".join(sentence)
            f.write(sentence_string + "\n")

In [8]:
def write_hdf5(filename):
    with open(res(out_path + 'input-sentences/' + filename + '.txt')) as file:
        elmo.embed_file(input_file=file, 
                        output_file_path = res(out_path + filename + '.hdf5'))

In [9]:
for border_size in border_sizes:
    train_sentences = get_sentences('train_original.txt', border_size)
    test_sentences = get_sentences('test_original.txt', border_size)
    write_sentences_to_txt(train_sentences, 'train_original_border_' + str(border_size))
    write_sentences_to_txt(test_sentences, 'test_original_border_' + str(border_size))
    write_hdf5('train_original_border_' + str(border_size))
    write_hdf5('test_original_border_' + str(border_size))

21233it [36:37,  9.66it/s]
4696it [09:17,  8.42it/s]
21233it [39:24,  8.98it/s]
4696it [07:33, 10.35it/s]
21233it [24:23, 14.51it/s]
4696it [05:17, 14.78it/s]
21233it [38:28,  9.20it/s]
4696it [08:25,  9.30it/s]
